In [1]:
import os
import pickle

import torch as th 
import numpy as np
import matplotlib.pyplot as plt
import torchvision;

from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

from scripts import *

import matplotlib

# save plots for latex
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# input
# model_checkpoint = best model pat
model_checkpoint = "experiments/dcgan_experiments/lightning_logs/version_2/checkpoints/epoch=974.ckpt"
save_path = "experiments/ganspace/dcgan_experiments_version_2_epoch=974"

# optional
n_samples = 5000
n_time = 5
layer="linear"

In [4]:
model = GAN.load_from_checkpoint(model_checkpoint)
model.eval()
1

1

In [5]:
@th.no_grad()
def get_linear_act(model, n_samples=10000, layer=None):
    z = th.normal(0, 1, size=(n_samples, model.generator.latent_dim), device=model.device)
    l = model.generator.l1(z)
    X = np.squeeze(l.detach().cpu().numpy())
    y = np.squeeze(z.detach().cpu().numpy())    
    return X, y

In [6]:
pca_path = os.path.join(save_path, layer, "pca.pkl")


if not os.path.exists(pca_path):
    pca = IncrementalPCA(n_components=4096 if layer == "linear" else 10)
    os.makedirs(os.path.join(save_path, layer), exist_ok=True)

    for i in range(n_time):
        print("Iteration - ", i+1)
        X, y = get_linear_act(model, n_samples, layer=layer)

        # save the data
        np.save(os.path.join(save_path, layer, f"y{i+1}.npy"), y)
        np.save(os.path.join(save_path, layer, f"X{i+1}.npy"), X)

        pca.partial_fit(X)

    # save the pca
    f = open(pca_path, "wb")
    pickle.dump(pca, f)
    f.close()
else:
    f = open(pca_path, 'rb')
    pca = pickle.load(f)
    f.close()

In [7]:
# load dataset
X = np.concatenate([np.load(os.path.join(save_path, layer, f"X{i+1}.npy")) for i in range(n_time)], axis=0)
y = np.concatenate([np.load(os.path.join(save_path, layer, f"y{i+1}.npy")) for i in range(n_time)], axis=0)

# information gain
information_gain = np.cumsum(pca.explained_variance_ratio_)
idx = np.where(information_gain >= 0.9)[0][0]

# 
comp = pca.components_[:idx]
mean = pca.mean_

# reconstruction
X_pca = (X - mean).dot(comp.T) 

# resonctruction error
X_rec = X_pca.dot(comp) + mean
np.linalg.norm(X_rec - X) / len(X)

0.057564076965052356

In [8]:
reg_path = os.path.join(save_path, layer, "reg.pkl")

if not os.path.exists(reg_path):
    reg = LinearRegression(normalize=True).fit(X_pca, y)
    f = open(reg_path, "wb")
    pickle.dump(reg, f)
    f.close()
else:
    f = open(reg_path, 'rb')
    reg = pickle.load(f)
    f.close()   
    
np.linalg.norm(X_pca.dot(reg.coef_.T) - y) / len(X)

0.052865363933402326

In [9]:
stds, means= [], []
for x in X_pca[:, :idx].T:
    means.append(x.mean())
    stds.append(x.std())
    
np.save(os.path.join(save_path, "pca_generator.npy"), {"E" : comp, "mu" : mean, "stds" : stds})

In [10]:
information_gain = np.cumsum(pca.explained_variance_ratio_)
plt.figure(figsize=(8, 4))
plt.bar(np.arange(2, 2*30+2, 2), 100*pca.explained_variance_ratio_[:30], width=1.4)
plt.xticks(np.arange(2, 2*30+2, 2), 1 + np.arange(30), fontsize=16, rotation=90)
plt.yticks(fontsize=16)
# plt.title("Explained Variance", fontsize=18)
plt.ylabel("Variance Percentage (%)", fontsize=18)
plt.xlabel("Principal Components", fontsize=18)
plt.grid(axis='y')
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.tight_layout()
plt.savefig(os.path.join(save_path, "pca_variance_percentage.pgf"))

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(/home/hca/.cache/matplotlib/tex.cache/265fdf92b077575fff48f2c6d04b2931.tex
LaTeX2e <2017-04-15>
Babel <3.18> and hyphenation patterns for 5 language(s) loaded.
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2014/09/29 v1.4h Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.6 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on 265fdf92b077575fff48f2c6d04b2931.log.




RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full report generated by latex:
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(/home/hca/.cache/matplotlib/tex.cache/265fdf92b077575fff48f2c6d04b2931.tex
LaTeX2e <2017-04-15>
Babel <3.18> and hyphenation patterns for 5 language(s) loaded.
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2014/09/29 v1.4h Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.6 \usepackage
               {type1ec}^^M
No pages of output.
Transcript written on 265fdf92b077575fff48f2c6d04b2931.log.




<Figure size 576x288 with 1 Axes>

In [11]:
#@th.no_grad()
#def explore_principle_direction(idx, U, model, zs, ratios=None):
#    
#    size = U.shape[1]
#    n_samples = zs.shape[0]
#    if ratios is None:
#        ratios = np.linspace(-0.5, 0.5, 11)
#   
#    x = th.zeros((size,), device=model.device, dtype=zs.dtype)
#    
#    imgs = []
#    for z in zs:
#        for ratio in ratios:
#            x[idx] = ratio
#            z_ = z + th.matmul(U, x)
#            
#            img = th.squeeze(model(th.unsqueeze(z_, dim=0)))
#            imgs.append(img)
#    
#    # make grid
#    grid = torchvision.utils.make_grid(imgs, nrow=len(ratios), normalize=True)
#    return grid
#
#
#@th.no_grad()
#def explore_principle_direction_pca(idx, E, mu, model, zs, ratios):
#    
#    n_samples = zs.shape[0]
#
#    
#    imgs = []
#    for z in zs:
#        for ratio in ratios:
#            l = model.l1(th.unsqueeze(z, dim=0))
#            l = th.squeeze(l)
#            l_pca = (l - mu).matmul(E.T)
#            l_pca[idx] = ratio
#            l_rec = l_pca.matmul(E) + mu
#            l_rec = l_rec.view(1,  model.init_channels,
#                    model.init_height,  model.init_width)
#            
#            img = th.squeeze(model.conv_blocks(l_rec))
#            imgs.append(img)
#        #print(2*l_pca[idx])
#    # make grid
#    grid = torchvision.utils.make_grid(imgs, nrow=len(ratios), normalize=True)
#    return grid
#
#
#def explore_pca(idx, E, mu, means, stds, model, ratios, cases):
#    imgs = []
#
#    for i in range(cases):
#        point = th.tensor([np.random.normal(mean, scale) for mean, scale in zip(means, stds)], device=model.device)
#        # point = th.tensor([0.0 for mean, scale in zip(means, stds)], device=model.device)
#        point[0] = max(point[0], 11.8467328) if point[0] > 0 else min(point[0], -11.8467328)
#
#        for ratio in ratios:
#            point[idx] = ratio
#            l_rec = point.matmul(E) + mu
#            l_rec = l_rec.view(1,  model.generator.init_channels,
#                    model.generator.init_height,  model.generator.init_width)
#            img = th.squeeze(model.generator.conv_blocks(l_rec))
#            imgs.append(img)
#            
#    grid = torchvision.utils.make_grid(imgs, nrow=len(ratios), normalize=True, pad_value=1)
#    return grid

@th.no_grad()
def explore_pca(gen, Z, idx, ratios):
    """Exploring PCA directions"""
    
    imgs = []
    for z in Z:
        for ratio in ratios:
            z[idx] = ratio
            img = th.squeeze(gen(z))
            imgs.append(img)
            
    grid = torchvision.utils.make_grid(imgs, nrow=len(ratios), normalize=True, pad_value=1)
    return grid

In [13]:
fig=plt.figure(figsize=(15, 15))
from scipy.stats import norm

columns = 6
rows = 5
for i in range(1, columns*rows +1):
    dim = X_pca[:, i-1]
    fig.add_subplot(rows, columns, i)
    
    # data dist
    plt.hist(dim, bins=50, density=True, alpha=0.6)
    
    # fit gaus
    mu, std = dim.mean(), dim.std()
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
    plt.plot(x, p, 'k--', linewidth=2)
    
    plt.title("Eigen Dim %d" % i, fontsize=22)
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()
plt.savefig(os.path.join(save_path, "eigen_dim_distributions.pgf"))

In [15]:
Z = th.tensor([[np.random.normal(0, scale) for scale in stds] for _ in range(30)], device=model.device)
for z in Z:
    z[0] = max(z[0], 11.8467328) if z[0] > 0 else min(z[0], -11.8467328)

In [16]:
E = th.tensor(comp, device=model.device, dtype=Z.dtype)
mu = th.tensor(mean, device=model.device, dtype=Z.dtype)
gen = PCAGenerator(model.generator, E, mu)

In [17]:
Z.shape

torch.Size([30, 30])

In [22]:
Z = th.tensor([[np.random.normal(0, scale) for scale in stds] for _ in range(30)], device=model.device)
grid = torchvision.utils.make_grid(gen(Z), nrow=10, normalize=True, pad_value=1)
torchvision.utils.save_image(grid, os.path.join(save_path, f"untruncated.png"))

for z in Z:
    z[0] = max(z[0], 11.8467328) if z[0] > 0 else min(z[0], -11.8467328)
    
    #indexes = {
    #      1:(3, 3), 
    #      2:(3*0.75, 3*0.75), 
    #      3:(3*0.75, 3*0.75), 
    #      5:(3*0.75, 3*0.75), 
    #      7:(3, 3),
    #      10:(3*2, 3*2),
    #      11:(3, 3),
    #      12:(3, 3),
    #      15:(6, 6),
    #      16:(9, 9),
    #      17:(9, 9),
    #      18 : (6, 6),
    #      20 : (0, 12),
    #      21 : (0, 15),
    #      23 : (9,9 ),
    #      24 : (0, 15),
    #      25 : (0, 15)
    #      }
    #for dim in indexes:
    #    s1, s2 = indexes[dim]
    #    z[dim] =  min(z[dim], stds[dim]*s2) if z[dim] > 0 else max(z[dim], -stds[dim]*s1)
grid = torchvision.utils.make_grid(gen(Z), nrow=10, normalize=True, pad_value=1)
torchvision.utils.save_image(grid, os.path.join(save_path, f"truncated.png"))

In [ ]:
# %matplotlib inliine
i = 0    
scale = 3*stds[i]
scale2 = 1
ratios = np.linspace(-scale * scale2, scale * scale2, 11)
grid = explore_pca(gen, Z.clone(), i, ratios)
torchvision.utils.save_image(grid, os.path.join(save_path, f"pca_dim_{i}.png"))
grid = grid.detach().cpu().numpy().transpose(1, 2, 0)

plt.figure(figsize=(25, 15))
plt.imshow(grid)
plt.xticks([1, 2, 3])
plt.yticks([])
plt.axis('off')

In [ ]:
grid = torchvision.utils.make_grid(gen(Z), nrow=10, normalize=True, pad_value=1)
torchvision.utils.save_image(grid, os.path.join(save_path, f"original_cars.png"))

In [ ]:
# %matplotlib inliine
for i in range(len(stds)):
    scale = 3*stds[i]
    scale2 = 1
    ratios = np.linspace(-scale * scale2, scale * scale2, 11)
    grid = explore_pca(gen, Z.clone(), i, ratios)
    torchvision.utils.save_image(grid, os.path.join(save_path, f"pca_dim_{i}.png"))

In [ ]:
Z = th.tensor([np.random.normal(0, scale) for scale in stds], device=model.device)
Z[0] = max(Z[0], 11.8467328) if Z[0] > 0 else min(Z[0], -11.8467328)
imgs = []
for i in range(len(stds)):
    scale = 3*stds[i]
    scale2 = 1
    for ratio in np.linspace(-scale * scale2, scale * scale2, 21):
        z = Z.clone()
        z[i] = ratio 
        imgs.append(th.squeeze(gen(z)))
grid = torchvision.utils.make_grid(imgs, nrow=21, normalize=True, pad_value=1)
torchvision.utils.save_image(grid, os.path.join(save_path, f"pca_dims_all_one.png"))

In [ ]:
# shape
i = [23, 20, 11, 5, 9]

# lighting
i = [10, 12, 16, 6, 14, 9]

# deformations
i = [17, 18, 25, 24, 14]

# color 
i = [7, 15, 8 , 14, 9]

# car type
i = [1, 2]


In [12]:
model.cuda()
E = th.tensor(comp, device=model.device, dtype=th.float32)
mu = th.tensor(mean, device=model.device, dtype=th.float32)
pca_generator = PCAGenerator(model.generator, E, mu)


indexes = {0:(3*0.3, 3*0.3), 
          1:(3, 3), 
          2:(3*0.75, 3*0.75), 
          3:(3*0.75, 3*0.75), 
          5:(3*0.75, 3*0.75), 
          7:(3, 3),
          10:(3*2, 3*2),
          11:(3, 3),
          12:(3, 3),
          15:(6, 6),
          16:(9, 9),
          17:(9, 9),
          18 : (6, 6),
          20 : (0, 12),
          21 : (0, 15),
          23 : (9,9 ),
          24 : (0, 15),
          25 : (0, 15)
          }

groups = {
         "shape" : [23, 20, 11, 5, 9],
         "lighting" : [10, 12, 16, 6, 14, 9],
         "deformations" : [17, 18, 25, 24, 14],
         "color" : [7, 15, 8 , 14, 9], 
         "car_type" : [1, 2],
         "nothing" : []}

unknown = [4, 22, 26, 27, 28, 29]
truncate = True
import os

for key in groups:
    dims = groups[key]
    
    path = os.path.join(save_path, "truncated" if truncate else "nontruncated", f"{key}_disabled")
    os.makedirs(path, exist_ok=True)


    with th.no_grad():
        for i in range(10):
            points = [[np.random.normal(0, scale) for scale in stds] for _ in range(1000)]
            points = th.tensor(points, device=model.device)
            
            if dims:
                points[:, dims] = 0
            if truncate:
                for point in points:
                    point[0] = max(point[0], stds[0]*3*0.3) if point[0] > 0 else min(point[0], -stds[0]*3*0.3)
                    # should we generate 
            imgs = pca_generator(points)
                             
            for j, img in enumerate(imgs):
                torchvision.utils.save_image(img, os.path.join(path, f"{i*1000 + j}.png"), normalize=True, range=(-1, 1))

In [ ]:
idx = [] # [6, 8, 9, 13, 14, 19, 22, 26, 27, 28, 29]

indexes = {0:(3*0.3, 3*0.3), 
          1:(3, 3), 
          2:(3*0.75, 3*0.75), 
          3:(3*0.75, 3*0.75), 
          4:(3, 3), 
          5:(3*0.75, 3*0.75), 
          7:(3, 3),
          10:(3*2, 3*2),
          11:(3, 3),
          12:(3, 3),
          15:(3*2, 3*2),
          16:(9, 9),
          17:(9, 9),
          18 : (6, 6),
          20 : (0, 3*4),
          21 : (0, 3*5),
          23 : (9,9 ),
          24 : (0, 15),
          25 : (0, 15)
          }
# model.cuda()
E = th.tensor(comp, device=model.device, dtype=th.float32)
mu = th.tensor(mean, device=model.device, dtype=th.float32)
pca_generator = PCAGenerator(model.generator, E, mu)
scales = stds
imgs = []

with th.no_grad():    
    point = th.tensor([np.random.normal(0, scale) for scale in scales], device=model.device)
    point[0] = max(point[0], 11.8467328 / 1) if point[0] > 0 else min(point[0], -11.8467328 / 1)
    # point[20:] = 0 # 12

    #l_rec = point.matmul(E) + mu
    #l_rec = l_rec.view(1,  model.generator.init_channels,
    #        model.generator.init_height,  model.generator.init_width)
    #img =th.squeeze(model.generator.conv_blocks(l_rec))
    img = th.squeeze(pca_generator(point))
    imgs.append(img)

    for _ in range(19):
        point2 = point.clone()
        for idx_ in indexes:
            if idx_ == 0:
                continue
            if np.random.uniform(0, 1) > 0.25:
                a, b = indexes[idx_]
                point2[idx_] = np.random.uniform(-a*scales[idx_]/2, b*scales[idx_]/2)
        for idx_ in idx:
            point2[idx_] = 0

        l_rec = point2.matmul(E) + mu
        l_rec = l_rec.view(1,  model.generator.init_channels,
                model.generator.init_height,  model.generator.init_width)
        img = th.squeeze(model.generator.conv_blocks(l_rec))
        imgs.append(img)
    
grid = torchvision.utils.make_grid(imgs, nrow=5, normalize=True, pad_value=1)
grid = grid.detach().cpu().numpy().transpose(1, 2, 0)

plt.figure(figsize=(25, 15))
plt.xticks([])
plt.yticks([])
plt.imshow(grid)

In [ ]:
idx = [9, 13]

indexes = {0:(3*0.3, 3*0.3), 
          1:(3, 3), 
          2:(1.5, 1.5), 
          3:(3*0.75, 3*0.75), 
          4:(3, 0), 
          5:(3, 3), 
          7:(3, 3),
          8:(3, 3),
          10:(3*1.75, 3*1.75),
          11:(3, 3),
          12:(3*1.5, 0),
          14:(6, 6),
         15:(3*1.5, 3*1.5),
         16:(9, 9),
         17:(9, 9),
         18 : (6, 6),
         19 : (9, 9)}
# model.cuda()
E = th.tensor(comp, device=model.device, dtype=th.float32)
mu = th.tensor(mean, device=model.device, dtype=th.float32)
scales = stds
imgs = []

with th.no_grad():    
    point = th.tensor([np.random.normal(0, scale) for scale in scales], device=model.device)
    point[0] = max(point[0], 11.8467328 / 1) if point[0] > 0 else min(point[0], -11.8467328 / 1)
    # point[20:] = 0 # 12

    l_rec = point.matmul(E) + mu
    l_rec = l_rec.view(1,  model.generator.init_channels,
            model.generator.init_height,  model.generator.init_width)
    img =th.squeeze(model.generator.conv_blocks(l_rec))
    imgs.append(img)

    for _ in range(19):
        point2 = point.clone()
        for idx_ in indexes:
            if idx_ == 0:
                continue
            if np.random.uniform(0, 1) > 0.5:
                a, b = indexes[idx_]
                point2[idx_] = np.random.uniform(-a*scales[idx_]/2, b*scales[idx_]/2)
        for idx_ in idx:
            point2[idx_] = 0

        l_rec = point2.matmul(E) + mu
        l_rec = l_rec.view(1,  model.generator.init_channels,
                model.generator.init_height,  model.generator.init_width)
        img = th.squeeze(model.generator.conv_blocks(l_rec))
        imgs.append(img)
    
grid = torchvision.utils.make_grid(imgs, nrow=5, normalize=True)
grid = grid.detach().cpu().numpy().transpose(1, 2, 0)

plt.figure(figsize=(25, 15))
plt.xticks([])
plt.yticks([])
plt.imshow(grid)

In [ ]:
img_dim = model.hparams.img_dim[1:]
hue = [-0.1, 0.1]
contrast = [1, 1.25]
brightness = [1, 2]
data_augs = [
    # no dat aug
    transforms.Compose([transforms.Resize(img_dim),
                        transforms.ToTensor(),
                        transforms.Normalize([0.5], [0.5])]),
    # horizontal flip only
    transforms.Compose([transforms.Resize(img_dim),
                        transforms.ToTensor(),
                        transforms.RandomHorizontalFlip(p=0.5),
                        transforms.RandomVerticalFlip(p=0.5),
                        transforms.Normalize([0.5], [0.5])]),
    # horizontal flip + hue
    transforms.Compose([transforms.Resize(img_dim),
                        transforms.ToTensor(),
                        transforms.RandomHorizontalFlip(p=0.5),
                        transforms.RandomVerticalFlip(p=0.5),
                        transforms.ColorJitter(hue=hue),
                        transforms.Normalize([0.5], [0.5])]),
    # horizontal flip + hue + contrast
    transforms.Compose([transforms.Resize(img_dim),
                        transforms.ToTensor(),
                        transforms.RandomHorizontalFlip(p=0.5),
                        transforms.RandomVerticalFlip(p=0.5),
                        transforms.ColorJitter(hue=hue, contrast=contrast),
                        transforms.Normalize([0.5], [0.5])]),
    # horizontal flip + hue + contrast + brightness
    transforms.Compose([transforms.Resize(img_dim),
                        transforms.ToTensor(),
                        transforms.RandomHorizontalFlip(p=0.5),
                        transforms.RandomVerticalFlip(p=0.5),
                        transforms.ColorJitter(
                            hue=hue, brightness=brightness, contrast=contrast),
                        transforms.Normalize([0.5], [0.5])]),
]

potsdam_dir = "../potsdam_data/potsdam_cars_corrected"
# data loaders
i = 3
n_samples = 1024
dataset = ImageFolder(potsdam_dir, transform=data_augs[i])
potsdam_cars_dataloader = DataLoader(dataset, batch_size=4300, shuffle=True, num_workers=4)
real,_ = next(iter(potsdam_cars_dataloader))
act1 = np.squeeze(vgg16_get_activation_maps(
                real, layer_idx=33, device="cuda:0", normalize_range=(-1, 1)).numpy())

In [ ]:
with th.no_grad():
    points = []
    for _ in range(n_samples):
        point = [np.random.normal(0, scale) for scale in stds]
        #point[0] = max(point[0], 11.8467328 / 2) if point[0] > 0 else min(point[0], -11.8467328 / 2)
        point[0] = 11.8467328 if point[0] > 0 else -11.8467328
        points.append(point)
    points = th.tensor(points, device=model.device)
    l_rec = points.matmul(E) + mu
    l_rec = l_rec.view(n_samples,  model.generator.init_channels,
            model.generator.init_height,  model.generator.init_width)
    fake =th.squeeze(model.generator.conv_blocks(l_rec))

act2 = np.squeeze(vgg16_get_activation_maps(
                fake,  layer_idx=33, device="cuda:0", normalize_range=(-1, 1)).numpy())

fid_score(act1, act2,  n_cases=n_samples,skip_vgg=True, memorized_fid=False)

In [ ]:
# norms inverse, replace inf values with 0
act1_norm = np.expand_dims(np.linalg.norm(act1, axis=1), 1)
act2_norm = np.expand_dims(np.linalg.norm(act2, axis=1), 1)
norms_inverse = 1 / (act2_norm.dot(act1_norm.T))

norms_inverse[np.isinf(norms_inverse)] = 0

# cos distance between two activations
cos_distances = (1 - norms_inverse*act2.dot(act1.T))
cos_distance = cos_distances.min(axis=1)  # min for each row
d = np.mean(cos_distance)

plt.hist(cos_distance)
plt.show()

jj = np.random.choice(len(fake), 50, replace=False)
#jj = np.argsort(cos_distance)[-50:]
#jj = np.argsort(cos_distance)[:50]
n = 10
imgs = []
for j in jj:
    idx = np.argsort(cos_distances[j])
    
    imgs.append(fake[j])
    for idx_ in idx[:n]:
        imgs.append(real[idx_])
        
grid = torchvision.utils.make_grid(imgs, nrow=n+1, normalize=True)
torchvision.utils.save_image(grid, "dcgan_random.png")

In [ ]:
real_mean = (real.mean(axis=0).detach().cpu().numpy().transpose(1, 2, 0) + 1) / 2
fake_mean = (fake.mean(axis=0).detach().cpu().numpy().transpose(1, 2, 0) + 1) / 2

plt.figure(figsize=(8,8))
plt.subplot(2, 1, 1)
plt.imshow(real_mean)
plt.subplot(2, 1, 2)
plt.imshow(fake_mean)
plt.tight_layout()

np.linalg.norm(real_mean - fake_mean)

In [ ]:
model_checkpoint = "experiments/dcgan_disc_double_params_padding_reflect_lr_scheduler/lightning_logs/version_0/checkpoints/epoch=899.ckpt"
save_path = "experiments/ganspace/dcgan_disc_double_params_padding_reflect_lr_scheduler_epoch=899"

# dim 1 : [-s, s] : white car
# dim 2 : [-s, s] : white van or green car
# dim 3 : [-s, s] : makes car longer but results get worse
# dim 4 : [-s/2, s/2] : move cars up and down
# dim 5 : [-s, 0] : adds top window
# dim 6 : [-s, s] : -s : makes car thicker and higlights its surrounding, s : makes car brighter color
# dim 7 : [-s, s] : -s : bright spots on right, s : bright spots on left
# dim 8 : [-s, s] : -s : makes it red/orange, s: makes it blue/purple
# dim 9 : [-s, s] : -s : makes it blue/pruple, s: makes it red/pink (looks nicer than dim 8)
# dim 10: [] : keep 0
# dim 11: [-s, s] : -s : birght spots on top, s: bright spots on bottom
# dim 12: [-s*0.75, s*0.5] : -s : makes image birghter, s : makes image darker
# dim 13: [-s*0.75: 0] : -s : makes car shorter
# dim 14 : keep 0
# dim 15 : [-s, s] : -s : bright spots on right, s : darker right
# dim 16 : [-s*0.75, s] : -s : makes car longer, s:makes car shorter
# dim 17 : [-s*0.75, s*0.75] : -s : makes bottom darker, s: makes bottom brighter
# dim 18 : [-s, s] : -s bends/rotates it counter clock wise, s : bends/rotates it clock wise
# dim 19 : [-s, s] : -s makes carish green, s: makes car redish and highlights the car lamps
# dim 20 : [-s , s] : -s : add car lamps on right, s : add car lamps on left (car orientation is important)
# dim 21 : 
# dim 22 : 
# dim 23 : 
# dim 24 : 
# dim 25 : [0, s] : s : bends the left part of the car
# dim 26 : 
# dim 27 :
# dim 28 :
# dim 29 :

In [ ]:
i = 3
scale = np.sqrt(pca.singular_values_)[i] * 0.5
ratios = np.linspace(-scale * 1, scale * 1, 11)
grid = explore_principle_direction_pca(i, E, mu, model.generator, zs, ratios)
grid = grid.detach().cpu().numpy().transpose(1, 2, 0)
plt.figure(figsize=(25, 15))
plt.imshow(grid)
plt.xticks([])
plt.yticks([])

In [ ]:
ratios = np.linspace(-scale * 1, scale * 1, 11)
grid = explore_principle_direction(i, U, model, zs, ratios)
grid = grid.detach().cpu().numpy().transpose(1, 2, 0)

plt.figure(figsize=(25, 15))
plt.imshow(grid)
plt.xticks([])
plt.yticks([])

In [ ]:
E = th.tensor(comp, device=model.device, dtype=th.float32)
mu = th.tensor(mean, device=model.device, dtype=th.float32)
scales = stds

# point = th.tensor([np.random.uniform(-scale/2, scale/2) for scale in scales], device=model.device)
point = th.tensor([np.random.normal(0, scale) for scale in scales], device=model.device)

point[0] = -1*-11.8467328
#point[13] = 0


l_rec = point.matmul(E) + mu
l_rec = l_rec.view(1,  model.generator.init_channels,
        model.generator.init_height,  model.generator.init_width)
img = th.squeeze(model.generator.conv_blocks(l_rec))
img = (img.detach().cpu().numpy().transpose(1, 2, 0 ) + 1) / 2
plt.imshow(img)

In [ ]:
point

In [ ]:
z = th.normal(0, 1, size=(1, model.generator.latent_dim), device=model.device) 
img = th.squeeze(model(z))
img = (img.detach().cpu().numpy().transpose(1, 2, 0 ) + 1) / 2
plt.imshow(img)

In [ ]:
np.sqrt(pca.singular_values_[50]), std

In [ ]:
zs = th.normal(0, 0.75, size=(15, model.generator.latent_dim), device=model.device) 
#zs = th.zeros((15, model.generator.latent_dim), device=model.device) 
U = th.tensor(reg.coef_, device=model.device, dtype=zs.dtype)

In [ ]:
# model

model_checkpoint = "experiments/dcgan_disc_double_params_padding_reflect_lr_scheduler/lightning_logs/version_0/checkpoints/epoch=899.ckpt"
save_path = "experiments/ganspace/dcgan_disc_double_params_padding_reflect_lr_scheduler_epoch=899"

# optional
n_samples = 5000
n_time = 5
layer = "linear"

# dim
i = 0, +/- 3*std*0.3, orientation of the car - look right + look left
i = 1, +/- 3*std, - van, + brown car
i = 2, +/- 3*std*0.5, 
i = 3, +/- 3*std*0.75, moves car up/down
i = 4, (-3*std, 0), adds top window
i = 5, (-3*std*0.75, 0) : makes car thicker and higlights its surrounding and adds strips on top
i = 6, 
i = 7, +/- 3*std, - makes car reddish and thicker, + makes car blueish and thinner
i = 8, +/- 3*std, - makes car blueish, + makes car reddish and add bright spot
i = 9, +/- 3*std, keep zero, doesn't have any results
i = 10, +/- 3*std*1.75, - bright spots on top, + bright spots on bottom
i = 11, +/- 3*std, - bright background, + dark background
i = 12, -3*std*1.5, - makes car shorter and brighter
i = 13, +/- 3*std, keep zero, doesn't have any results
i = 14, +/- 3*std*2, - bright spot and window on the top, + darker right
i = 15, +/- 3*std*1.5, - makes car longer, + makes car shorter
i = 16, +/- 3*std*3, idk something with light
i = 17, +/- 3*std*3, - bends/rotates it counter clock wise, + bends/rotates it clock wise
i = 18, +/- 3*std*2, - makes car green, + makes car redish and higlights the lamps
i = 19, +/- 3*std*3, - add car lamps on right and makes right side flat, + add car lamps on left and makes left side flat (car orientation is important)
i = 20, 
i = 21, +/- 3*std*4, - makes car gren, + makes car pink
i = 22,
i = 23, 
i = 24, 
i = 25,
i = 26,
i = 27,

In [ ]:
# input
model_checkpoint = "experiments/dcgan_resnet/lightning_logs/version_0/checkpoints/epoch=524.ckpt"
save_path = "experiments/ganspace/dcgan_resnet_version_0_epoch=524"

model = GAN.load_from_checkpoint(model_checkpoint)
model.eval()

# optional
n_samples = 5000
n_time = 5
layer = "linear"

# dim 


dims_ = [0, 1, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 19, 20]
ratios_ = [[-50, 50], [-30, 30], 
          [-50, 50], [-50, 50], 
          [-50, 0], [0, 50], 
          [-50, 0], [-50, 50], 
          [0, 40], [-40 , 40], 
          [0, 50], [-50, 50], 
          [-70, 0], [-50, 50]]

np.save(os.path.join(save_path, layer, "important_dims.npy"), {"dims" : dims_, "ratios" : ratios_})

zs = th.zeros((1, model.generator.latent_dim), device=model.device) 
U = th.tensor(reg.coef_, device=model.device, dtype=zs.dtype)

grids = []
for d, (down, up) in zip(dims_, ratios_):
    ratios = np.linspace(down, up, 11)
    grids.append(explore_principle_direction(d, U, model, zs, ratios))
    
grid = torchvision.utils.make_grid(grids, nrow=1, normalize=True)
torchvision.utils.save_image(grid, os.path.join(save_path, layer, "pca_directions.png"))

zs = th.normal(0, 0.75, size=(15, model.generator.latent_dim), device=model.device) 
U = th.tensor(reg.coef_, device=model.device, dtype=zs.dtype)

grids = []
for d, (down, up) in zip(dims_, ratios_):
    ratios = np.linspace(down, up, 11)
    grid = explore_principle_direction(d, U, model, zs, ratios)
    torchvision.utils.save_image(grid, os.path.join(save_path, layer, f"pca_dim={d}.png"))


In [ ]:
# input
model_checkpoint = "experiments/big_image/lightning_logs/version_0/checkpoints/epoch=449.ckpt"
save_path = "experiments/ganspace/big_image_version_0_epoch=449"

model = GAN.load_from_checkpoint(model_checkpoint)
model.eval()

# optional
n_samples = 5000
n_time = 5
layer = "linear"

# dim 0 : ratio [-50, 50] : make cars white and them left or right
# dim 1 : ratio [-30, 30] : makes cars white van or green
# dim 5 : ratio [-50, 50] : car becomes red or blue
# dim 6 : ratio [-50, 50] : makes car blue or redish with strips on top
# dim 7 : ratio [-50, 0] : adds bright spot in right side
# dim 8 : ratio [0, 50] : adds bright spot in left side
# dim 9 : ratio [-50, 0] : adds bright spot in left side
# dim 10 : ratio [-50, 50] : makes image brighter or darker
# dim 12 : ratio  [0, 40] : tries to add roof
# dim 11 : ratio [-40, 40] : makes bottom part brighter or top part darker
# dim 14 : ratio [0, 50] : makes car longer
# dim 15 : ratio [-50, 50] : adds car light bottom left or bends it to bottom right
# dim 19 : ratio [-70, 0] : bends it to bottom left
# dim 20 : ratio [-50, 50] : makes car smaller or rotates the car


dims_ = [0, 1, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 19, 20]
ratios_ = [[-50, 50], [-30, 30], 
          [-50, 50], [-50, 50], 
          [-50, 0], [0, 50], 
          [-50, 0], [-50, 50], 
          [0, 40], [-40 , 40], 
          [0, 50], [-50, 50], 
          [-70, 0], [-50, 50]]

np.save(os.path.join(save_path, layer, "important_dims.npy"), {"dims" : dims_, "ratios" : ratios_})

zs = th.zeros((1, model.generator.latent_dim), device=model.device) 
U = th.tensor(reg.coef_, device=model.device, dtype=zs.dtype)

grids = []
for d, (down, up) in zip(dims_, ratios_):
    ratios = np.linspace(down, up, 11)
    grids.append(explore_principle_direction(d, U, model, zs, ratios))
    
grid = torchvision.utils.make_grid(grids, nrow=1, normalize=True)
torchvision.utils.save_image(grid, os.path.join(save_path, layer, "pca_directions.png"))

zs = th.normal(0, 0.75, size=(15, model.generator.latent_dim), device=model.device) 
U = th.tensor(reg.coef_, device=model.device, dtype=zs.dtype)

grids = []
for d, (down, up) in zip(dims_, ratios_):
    ratios = np.linspace(down, up, 11)
    grid = explore_principle_direction(d, U, model, zs, ratios)
    torchvision.utils.save_image(grid, os.path.join(save_path, layer, f"pca_dim={d}.png"))


In [ ]:
# input
model_checkpoint = "experiments/dataaug/lightning_logs/version_1/checkpoints/epoch=774.ckpt"
save_path = "experiments/ganspace/dataaug_bn_default_epoch=774"

model = GAN.load_from_checkpoint(model_checkpoint)
model.eval()

# optional
n_samples = 5000
n_time = 5
layer = "linear"

# dim 0 : ratio [-50, 50] : make cars white and them left or right
# dim 1 : ratio [-20, 20] : makes cars white van or green
# dim 3 : ratio [-20, 20] : moves car from top to bottom
# dim 5 : ratio [-40, 40] : car becomes red or gets strips on top
# dim 6 : ratio [-30, 0] : makes car blue
# dim 9 : ratio [-20, 20] : makes image brighter or darker
# dim 10 : ratio [-40, 0] : adds bright spot and makes car green
# dim 11 : ratio [-40, 40] : makes image darker and green or blue and bright spot on bottom
# dim 13 : ratio [-30, 30] : makes image red or green
# dim 14 : ratio [-60, 0] : adds car light front and back
# dim 16 : ratio [-60, 60] : lighting direction
# dim 17 : ratio [-50, 50] : changes the aspect ratio of car + lighting on front or back
# dim 20 : ratio [-40, 40] : twists the car and rotates little bit and adds light spot
# dim 25 : ratio [-70, 70] : makes car green or red and add bright spot on left or right
# dim 44 : ratio [-100, 0] : bending the car

dims_ = [0, 1, 3, 5, 6, 9, 10, 11, 13, 14, 16, 17, 20, 25, 44]
ratios = [[-50, 50], [-20, 20], 
          [-20, 20], [-40, 40], 
          [-30, 0], [-20, 20], 
          [-40, 0], [-40, 40], 
          [-30, 30], [-60 , 0], 
          [-60, 60], [-50, 50], 
          [-40, 40], [-70, 70], 
          [-100, 0]]

np.save(os.path.join(save_path, layer, "important_dims.npy"), {"dims" : dims_, "ratios" : ratios_})

zs = th.zeros((1, model.generator.latent_dim), device=model.device) 
U = th.tensor(reg.coef_, device=model.device, dtype=zs.dtype)

grids = []
for d, (down, up) in zip(dims_, ratios_):
    ratios = np.linspace(down, up, 11)
    grids.append(explore_principle_direction(d, U, model, zs, ratios))
    
grid = torchvision.utils.make_grid(grids, nrow=1, normalize=True)
torchvision.utils.save_image(grid, os.path.join(save_path, layer, "pca_directions.png"))

zs = th.normal(0, 0.75, size=(15, model.generator.latent_dim), device=model.device) 
U = th.tensor(reg.coef_, device=model.device, dtype=zs.dtype)

grids = []
for d, (down, up) in zip(dims_, ratios_):
    ratios = np.linspace(down, up, 11)
    grid = explore_principle_direction(d, U, model, zs, ratios)
    torchvision.utils.save_image(grid, os.path.join(save_path, layer, f"pca_dim={d}.png"))
